In [1]:
import Pkg; Pkg.activate("../test")

  Activating project at `~/g/AlgebraicJulia/Gatlab.jl/test`


Catlab is supposed to be a computer algebra system; I've been working on the
actual computer algebra capabilities, in a package called Gatlab (Generalized
Algebraic Theories)

What is the difference between:
- MathML
- Symbolics/SciML expressions
- GAT expressions
?

First of all, MathML is a serialization/presentation format. In order to work
with it in a programming language, we need datastructures and libraries that
allow us to actually access the semantic meaning of the expressions.

Symbolics/SciML expressions do a great job of this, but make some design choices
that make them not ideal for our uses. Namely, for reasons having to do with
variable scoping, a SciML model is really like a "model factory", which creates
fresh variables on every invocation. The problem is that this, being an arbitrary
Julia function, isn't introspectable/serializable.

Additionally, the tight integration with Julia and the ability to interface with
arbitrary Julia code means that while import from MathML to SciML expressions
has been implemented, SciML to MathML is significantly trickier.

This isn't a bad choice for what SciML does most of the time, but it imposes
limitations for our vision of "serializable models." Being able to "roundtrip"
serialization is important for implementing model manipulation operations.

So you can think of Gatlab as a "serializable-first" computer algebra system,
built from the ground up to work with data structures for symbolic computing
that are interoperable between different programming languages, just like how
acsets are now serializable between Python and Julia.

Of course, there are still many uses for Symbolics.jl and MathML expressions.
MathML is a good serialization format for the expressions themselves, and
Symbolics.jl is a must to integrate with all of the fantastic SciML ecosystem
libraries, so Gatlab converts bidirectionally with fragments of MathML and
Symbolics.jl (i.e., Gatlab -> MathML, Gatlab -> SciML always works, but
arbitrary MathML/Symbolics.jl expressions may not be well-formed enough to be
converted).

In [2]:
using Revise, ModelingToolkit, Gatlab

[ Info: Precompiling Gatlab [f0ffcf3b-d13a-433e-917c-cc44ccf5ead2]
[ Info: Precompiling GatlabSciMLExt [15878fac-8bdd-50c2-a9e9-8cf61912daeb]


In [3]:
@variables a, b

2-element Vector{Num}:
 a
 b

In [10]:
expr = a + 2.5b

a + 2.5b

In [5]:
GatlabSciMLExt = Base.get_extension(Gatlab, :GatlabSciMLExt)

GatlabSciMLExt

In [6]:
genmap = Dict(a => ATrmCon(Lvl(1; context=true)), b => ATrmCon(Lvl(2; context=true)))

Dict{Num, ATrmCon} with 2 entries:
  a => ATrmCon({1}, ATrm[])
  b => ATrmCon({2}, ATrm[])

In [11]:
GatlabSciMLExt.convert_term(
    expr.val,
    GatlabSciMLExt.ELEMENTARY_FUNCTION_OPMAP,
    genmap,
    ATrm;
    interp_val=(x -> interp_val(AThRealAlg(), x))
)

ATrmCon(2, ATrm[ATrmCon({1}, ATrm[]), ATrmCon(11, ATrm[Const(ATyp(1, ATrm[]), 2.5), ATrmCon({2}, ATrm[])])])